In [3]:
import os
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
def convert_to_dataframe(path):
    data = []
    subfolder_column = []
    subsubfolder_column = []

    for subfolder, subsubfolders, files in os.walk(path):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(subfolder, file)
                with open(file_path, 'r') as f:
                    json_data = json.load(f)
                nested_df = json_normalize(json_data)
                data.append(nested_df)
                subfolder_column.extend([os.path.basename(subfolder)] * len(nested_df))
                subsubfolder_column.extend([os.path.basename(os.path.dirname(subfolder))] * len(nested_df))

    df = pd.concat(data, ignore_index=True)
    df['subfolder'] = subfolder_column
    df['subsubfolder'] = subsubfolder_column
    return df


df = convert_to_dataframe(r'C:/Users/user/PycharmProjects/phonepe/data/aggregated/user/country/india/state')
df.head()

<ipython-input-3-6035d61e5967>:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nested_df = json_normalize(json_data)


,success,code,responseTimestamp,data.aggregated.registeredUsers,data.aggregated.appOpens,data.usersByDevice,subfolder,subsubfolder
0,True,SUCCESS,1630501494543,6740,0,"[{'brand': 'Xiaomi', 'count': 1665, 'percentag...",2018,andaman-&-nicobar-islands
1,True,SUCCESS,1630501494939,9405,0,"[{'brand': 'Xiaomi', 'count': 2303, 'percentag...",2018,andaman-&-nicobar-islands
2,True,SUCCESS,1630501495385,12149,0,"[{'brand': 'Xiaomi', 'count': 2950, 'percentag...",2018,andaman-&-nicobar-islands
3,True,SUCCESS,1630501495714,15222,0,"[{'brand': 'Xiaomi', 'count': 3719, 'percentag...",2018,andaman-&-nicobar-islands
4,True,SUCCESS,1630501496432,18596,0,"[{'brand': 'Xiaomi', 'count': 4576, 'percentag...",2019,andaman-&-nicobar-islands


In [4]:
df.rename(columns = {"data.usersByDevice" : "usersByDevice"}, inplace = True)
df.head()

,success,code,responseTimestamp,data.aggregated.registeredUsers,data.aggregated.appOpens,usersByDevice,subfolder,subsubfolder
0,True,SUCCESS,1630501494543,6740,0,"[{'brand': 'Xiaomi', 'count': 1665, 'percentag...",2018,andaman-&-nicobar-islands
1,True,SUCCESS,1630501494939,9405,0,"[{'brand': 'Xiaomi', 'count': 2303, 'percentag...",2018,andaman-&-nicobar-islands
2,True,SUCCESS,1630501495385,12149,0,"[{'brand': 'Xiaomi', 'count': 2950, 'percentag...",2018,andaman-&-nicobar-islands
3,True,SUCCESS,1630501495714,15222,0,"[{'brand': 'Xiaomi', 'count': 3719, 'percentag...",2018,andaman-&-nicobar-islands
4,True,SUCCESS,1630501496432,18596,0,"[{'brand': 'Xiaomi', 'count': 4576, 'percentag...",2019,andaman-&-nicobar-islands


In [5]:
def unnest_dict(df, column_name):
    new_columns = set()
    for i, row in df.iterrows():
        nested_list = row[column_name]
        if nested_list is not None:
            for nested_dict in nested_list:
                for key in nested_dict.keys():
                    new_columns.add(key)
        else:
            for key in new_columns:
                row[key] = "0"
    for key in new_columns:
        df[key] = None
    
    for i, row in df.iterrows():
        nested_list = row[column_name]
        if nested_list is not None:
            for nested_dict in nested_list:
                for key, value in nested_dict.items():
                    row[key] = value
        else:
            for key in new_columns:
                row[key] = "0"
        df.loc[i] = row
    df = df.drop(columns=[column_name])
    return df

df= unnest_dict(df, 'usersByDevice')

In [4]:
df.head()

,success,code,responseTimestamp,data.aggregated.registeredUsers,data.aggregated.appOpens,subfolder,subsubfolder,brand,count,percentage
0,True,SUCCESS,1630501494543,6740,0,2018,andaman-&-nicobar-islands,Others,684,0.101484
1,True,SUCCESS,1630501494939,9405,0,2018,andaman-&-nicobar-islands,Others,916,0.097395
2,True,SUCCESS,1630501495385,12149,0,2018,andaman-&-nicobar-islands,Others,1145,0.094246
3,True,SUCCESS,1630501495714,15222,0,2018,andaman-&-nicobar-islands,Others,1372,0.090133
4,True,SUCCESS,1630501496432,18596,0,2019,andaman-&-nicobar-islands,Others,1664,0.089482


In [6]:
df.drop(columns = ['success', 'code', 'responseTimestamp', 'data.aggregated.appOpens'], inplace = True)

In [7]:
df.rename(columns = {'data.aggregated.registeredUsers':'registeredUsers','subfolder':'transaction_year', 'subsubfolder' : 'states', 'count': 'counts', 'percentage':'percent'}, inplace = True)

In [7]:
df.head()

,registeredUsers,transaction_year,states,brand,counts,percent
0,6740,2018,andaman-&-nicobar-islands,Others,684,0.101484
1,9405,2018,andaman-&-nicobar-islands,Others,916,0.097395
2,12149,2018,andaman-&-nicobar-islands,Others,1145,0.094246
3,15222,2018,andaman-&-nicobar-islands,Others,1372,0.090133
4,18596,2019,andaman-&-nicobar-islands,Others,1664,0.089482


In [8]:
import numpy as np

df = df.sort_values(by=['states', 'transaction_year'])
df['counter'] = df.groupby(['states', 'transaction_year']).cumcount() + 1
df['quarter'] = df['counter'].map({1: 'quarter1', 2: 'quarter2', 3: 'quarter3', 4: 'quarter4'})


In [9]:
df.to_csv('aggregateduser.csv', index=False)

In [10]:
df.drop(columns = ['quarter'], inplace = True)
df.rename(columns = {'states': 'States'})
df.rename(columns = {'counter': 'Quarters', 'registeredUsers': 'Registered_Users', 'transaction_year': 'Transaction_Year', 'brand' :'Brand', 'counts': 'User_Counts', 'percent' : 'User_Percentage'}, inplace = True)

In [11]:
# df.rename(columns = {'percent': 'percentages'}, inplace = True)
df.head()

,Registered_Users,Transaction_Year,states,User_Counts,User_Percentage,Brand,Quarters
0,6740,2018,andaman-&-nicobar-islands,684,0.101484,Others,1
1,9405,2018,andaman-&-nicobar-islands,916,0.097395,Others,2
2,12149,2018,andaman-&-nicobar-islands,1145,0.094246,Others,3
3,15222,2018,andaman-&-nicobar-islands,1372,0.090133,Others,4
4,18596,2019,andaman-&-nicobar-islands,1664,0.089482,Others,1


In [13]:
df.rename(columns = {'states': 'States'}, inplace = True)
df.head(500)

,Registered_Users,Transaction_Year,States,User_Counts,User_Percentage,Brand,Quarters
0,6740,2018,andaman-&-nicobar-islands,684,0.101484,Others,1
1,9405,2018,andaman-&-nicobar-islands,916,0.097395,Others,2
2,12149,2018,andaman-&-nicobar-islands,1145,0.094246,Others,3
3,15222,2018,andaman-&-nicobar-islands,1372,0.090133,Others,4
4,18596,2019,andaman-&-nicobar-islands,1664,0.089482,Others,1
...,...,...,...,...,...,...,...
495,259990,2021,nagaland,27973,0.107593,Others,4
496,276483,2022,nagaland,29371,0.106231,Others,1
497,292104,2022,nagaland,0,0,0,2
498,307386,2022,nagaland,0,0,0,3


In [14]:
df.head(500)

,Registered_Users,Transaction_Year,States,User_Counts,User_Percentage,Brand,Quarters
0,6740,2018,andaman-&-nicobar-islands,684,0.101484,Others,1
1,9405,2018,andaman-&-nicobar-islands,916,0.097395,Others,2
2,12149,2018,andaman-&-nicobar-islands,1145,0.094246,Others,3
3,15222,2018,andaman-&-nicobar-islands,1372,0.090133,Others,4
4,18596,2019,andaman-&-nicobar-islands,1664,0.089482,Others,1
...,...,...,...,...,...,...,...
495,259990,2021,nagaland,27973,0.107593,Others,4
496,276483,2022,nagaland,29371,0.106231,Others,1
497,292104,2022,nagaland,0,0,0,2
498,307386,2022,nagaland,0,0,0,3


In [15]:
import pyodbc

conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=ASHWADHAASINI\SQLEXPRESS;'
                      'Database=PHONEPE;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()

In [15]:
cursor.execute('''
    CREATE TABLE Aggregateduserdatatouse (
        States nvarchar(50),
        Transaction_Year int,
        Quarters int,
        Registered_Users int,
        User_Percentage decimal,
        User_Counts int,
        Brand nvarchar(50))
               ''')


In [16]:
for row in df.itertuples():
    cursor.execute('''
                INSERT INTO Aggregateduserdatatouse (States, Transaction_Year, Quarters, Registered_Users, User_Percentage, User_Counts, Brand)
                VALUES (?,?,?,?,?,?,?)
                ''',
                row.States, 
                row.Transaction_Year,
                row.Quarters,
                row.Registered_Users,
                row.User_Percentage,
                row.User_Counts,
                row.Brand
                )
conn.commit()

In [18]:
cursor.execute("SELECT * FROM dbo.Aggregateduserdatatouse")
rows = cursor.fetchall()
for row in rows:
    print(row)

('andaman-&-nicobar-islands', 2018, 1, 6740, Decimal('0'), 684, 'Others')
('andaman-&-nicobar-islands', 2018, 2, 9405, Decimal('0'), 916, 'Others')
('andaman-&-nicobar-islands', 2018, 3, 12149, Decimal('0'), 1145, 'Others')
('andaman-&-nicobar-islands', 2018, 4, 15222, Decimal('0'), 1372, 'Others')
('andaman-&-nicobar-islands', 2019, 1, 18596, Decimal('0'), 1664, 'Others')
('andaman-&-nicobar-islands', 2019, 2, 21731, Decimal('0'), 1981, 'Others')
('andaman-&-nicobar-islands', 2019, 3, 23931, Decimal('0'), 2110, 'Others')
('andaman-&-nicobar-islands', 2019, 4, 25960, Decimal('0'), 2183, 'Others')
('andaman-&-nicobar-islands', 2020, 1, 28368, Decimal('0'), 2257, 'Others')
('andaman-&-nicobar-islands', 2020, 2, 32710, Decimal('0'), 2617, 'Others')
('andaman-&-nicobar-islands', 2020, 3, 41335, Decimal('0'), 3362, 'Others')
('andaman-&-nicobar-islands', 2020, 4, 52396, Decimal('0'), 4243, 'Others')
('andaman-&-nicobar-islands', 2021, 1, 57758, Decimal('0'), 4532, 'Others')
('andaman-&-nico

In [16]:
df = pd.read_sql('SELECT * FROM dbo.Aggregateduserdatatouse', con=conn)

In [88]:
query = "UPDATE dbo.Aggregateduserdatatouse SET States = 'Uttarakhand' WHERE States = 'uttarakhand'"
cursor = conn.cursor()
cursor.execute(query)
conn.commit()

In [89]:
cursor.execute("SELECT * FROM dbo.Aggregateduserdatatouse")
rows = cursor.fetchall()
for row in rows:
    print(row)

('Andaman & Nicobar Island', 2018, 1, 6740, Decimal('0'), 684, 'Others')
('Andaman & Nicobar Island', 2018, 2, 9405, Decimal('0'), 916, 'Others')
('Andaman & Nicobar Island', 2018, 3, 12149, Decimal('0'), 1145, 'Others')
('Andaman & Nicobar Island', 2018, 4, 15222, Decimal('0'), 1372, 'Others')
('Andaman & Nicobar Island', 2019, 1, 18596, Decimal('0'), 1664, 'Others')
('Andaman & Nicobar Island', 2019, 2, 21731, Decimal('0'), 1981, 'Others')
('Andaman & Nicobar Island', 2019, 3, 23931, Decimal('0'), 2110, 'Others')
('Andaman & Nicobar Island', 2019, 4, 25960, Decimal('0'), 2183, 'Others')
('Andaman & Nicobar Island', 2020, 1, 28368, Decimal('0'), 2257, 'Others')
('Andaman & Nicobar Island', 2020, 2, 32710, Decimal('0'), 2617, 'Others')
('Andaman & Nicobar Island', 2020, 3, 41335, Decimal('0'), 3362, 'Others')
('Andaman & Nicobar Island', 2020, 4, 52396, Decimal('0'), 4243, 'Others')
('Andaman & Nicobar Island', 2021, 1, 57758, Decimal('0'), 4532, 'Others')
('Andaman & Nicobar Island', 